# Patch

### **Exercise** (Patch Data)
Calculate and print following:

1. The number of samples in the data
2. The minimum and maximum value of the data

In [ ]:
array.size

array.max()
array.min()

### **Exercise** (Patch Attrs)
Do the following:

1. Print the cable_id from the patch with updated attrs.
2. Create a new patch with a station name of "DAS1".

In [ ]:
print(patch_updated_attrs.attrs["cable_id"])
patch_updated_attrs = patch_updated_attrs.update_attrs(station="DAS1")

### **Exercise** (Patch Coords)

Calculate the and print following parameters:

1. The duration (time) of the patch recording using the time coordinate.

2. Reset the start of the time coordiante to noon on the release date of Elton John's single Rocket Man (17 April 1972)


In [ ]:
duration = time_s.max() - time_s.min()
duration

time_coord = patch.get_coord("time")
t_min = "1972-04-17" 
patch_new_time = time_coord.update(min=t_min)
patch_new_time

### **Exercise** (Patch Select)
Remove the first 10 spatial channels then the last .05 seconds. 

In [ ]:
sub_patch = patch.select(distance=(10, None), samples=True).select(time=(None, -0.05), relative=True)
sub_patch

### **Exercise** (Transformation)

Do the following to the patch above:

1. Perform a real discrete Fourier transform along the time axis
2. Select frequencies (`ft_time` dimension) less than 500 Hz
3. Take the mean along the distance dimension
4. Get the amplitude spectra
5. Plot using matplotlib and the `ft_time` dimension's values


In [ ]:
import matplotlib.pyplot as plt

amplitude = patch_fft.data

sub_dft = patch_fft.select(ft_time=(None, 500))
mean_dft = sub_dft.aggregate("distance")
amp = np.abs(mean_dft.squeeze().data)

freq = sub_dft.get_array("ft_time")

plt.plot(freq, amp)
plt.xlabel("ft_time [Hz]")
plt.ylabel("Amplitude")
plt.title("Amplitude vs ft_time")
plt.show()

# Spool


### **Exercise** (Spool 1)

Using the diverse das spool, determine how many unique stations are represented. 

In [ ]:
diverse_spool = dc.get_example_spool("diverse_das")

contents_df = diverse_spool.get_contents()
stations = contents_df["station"]
stations

### **Exercise** (Spool 2)

Sort the diverse spool based on time (using [`Spool.sort`'](https://dascore.org/api/dascore/core/spool/DataFrameSpool/sort.html)), create a sub-spool with the last 4 patches. Print the attrs of each patch in this spool. 

In [ ]:
sorted_sub_spool = diverse_spool.sort("time")[:4]
for patch in sorted_sub_spool:
    print(patch.attrs)

### **Exercise** (Spool 3)

Create a sub-spool of the diverse spool by selecting all patches with a station code that ends with an 's'. 

In [ ]:
sub_spool = diverse_spool.select(station="*s")
sub_spool

### **Exercise** (Spool 4)

Do the following: 

1. Chunk the diverse spool to have 8 second patches (no partial patch needed) with no overlaps.
2. Combine all compatible patches along the time dimension. 

Determine how many patches are in these new spools.

In [ ]:
chunked_spool = diverse_spool.chunk(time=8, keep_partial=False)
print(len(chunked_spool))

merged_spool = diverse_spool.chunk(time=None)
print(len(merged_spool))

# Application

### **Exercise** 
Use `Patch.select` to remove the channels outside of study area.

In [ ]:
sub_patch = patch.select(distance=(50, 550)).taper(time=0.1)

### Exercise 
Using the `Patch.pass_filter()` funtion, apply a band-pass filter of 1 to 30 Hz. Then, visualize the f-k plot.

In [ ]:
pa_bandpassed = pa_down_sampled.pass_filter(time=(1*Hz, 30*Hz)).viz.spectrogram(scale=0.1)

### **Exercise** 
Use `Patch.rolling` to improve denoising the data by applying a moving median over 0.1 second window with 0.5 step.
Then, visualize and save the processed data.

In [ ]:
patch_median_filtered = patch_slope_filtered.rolling(time=0.1).median()
patch_median_filtered.viz.waterfall(scale=0.01)